<a href="https://colab.research.google.com/github/l-87hjl/3i-atlas-public-data/blob/main/00_scrape_mpec_observations_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Outputs

### 1. `observations_MPEC.csv` (Full Data)
Complete observation records with all available fields:
- `timestamp`: UTC observation time in `YYYY-MM-DD HH:MM:SS.ssssss` format
  - **Zero-padded hours**: `00:10:29`, NOT `0:10:29`
  - **Microsecond precision**: 6 decimal places (e.g., `.123456`)
  - **Derived from MPC fractional day** with maximum available precision
- `observatory`: MPC 3-character observatory code (e.g., "W50", "033", "G96")
  - **MUST be string type** to preserve leading zeros
- `observatory_name`: Full observatory name (e.g., "Whipple Observatory")
- `obs_ra`: Right Ascension in `HHMMSS.SSS` format (J2000.0, packed sexagesimal)
  - **Zero-padded hours**: `104713.073`, NOT `14713.073`
  - Maximum available precision from MPC (typically 3 decimal places)
- `obs_dec`: Declination in `±DD MM SS.SS` format (J2000.0, space-separated sexagesimal)
  - **Zero-padded degrees**: `+07 00 13.39`, NOT `+7 0 13.39`
  - Maximum available precision from MPC (typically 2 decimal places)
- `magnitude`: Apparent magnitude (if reported, otherwise null)
- `reference`: MPEC reference identifier (e.g., "MPEC Y51")

### 2. `observations_timestamp_observatory_only.csv` (Minimal Index)
Lightweight index file containing only:
- `timestamp`: UTC observation time (same format as above: `YYYY-MM-DD HH:MM:SS.ssssss`)
- `observatory`: MPC 3-character observatory code (string type)

**Used for JPL Horizons ephemeris queries** - these are the only two fields needed to request observer-specific ephemeris data.

---


In [1]:
# Install required packages
!pip install -q requests beautifulsoup4 pandas lxml

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime, timedelta
import time
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


---

## 🚨 CRITICAL: How to Verify Your CSV Files Are Correct

### ⚠️ The "Excel Problem"

**If you open the CSV file in Excel or Google Sheets, you may see:**
- `0:35:55` instead of `00:35:55` (missing leading zero) ❌
- `12:35:55 AM` instead of `00:35:55.507200` (converted to time format) ❌
- `33` instead of `"033"` (stripped leading zero from observatory code) ❌
- Missing subsecond precision ❌

**BUT THE CSV FILE ITSELF IS CORRECT!** ✅

### ✅ How to Verify Your Files Are Actually Correct:

**Option 1: Open in a text editor** (Notepad, TextEdit, VS Code, Sublime)
```
timestamp,observatory,obs_ra,obs_dec
2025-12-19 00:10:29.123456,033,104713.073,+07 00 13.39  ← SEE THE LEADING ZEROS!
```

**Option 2: Use Python/pandas**
```python
import pandas as pd
df = pd.read_csv('your_file.csv', dtype={'observatory': str})
print(df['timestamp'].iloc[0])  # Will show: 2025-12-19 00:10:29.123456
```

**Option 3: Use the verification cell** (included in this notebook below)
- After saving files, run the verification cell
- It will prove character-by-character that the format is correct

### 📊 Summary

- **The scraper produces correct output** ✅
- **Excel/Sheets visually destroy the display** ❌
- **Always verify in text editor or Python** ✅

---


## Configuration & Helper Functions

In [3]:
# MPC 3I/ATLAS permalink
MPC_URL = "https://minorplanetcenter.net/db_search/show_object?utf8=%E2%9C%93&object_id=3I"

# Headers for polite scraping
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Scientific Research; 3I/ATLAS Analysis) AppleWebKit/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
}

def normalize_ra_format(ra_str: str) -> str:
    """
    Convert RA from various formats to packed HHMMSS.SSS format (no spaces/colons).

    Preserves maximum precision from original MPC data.

    Args:
        ra_str: Right Ascension in format "HH:MM:SS.SS" or "HH MM SS.SS"

    Returns:
        RA in packed format "HHMMSS.SSS" (J2000)
    """
    ra_str = ra_str.strip().replace(':', ' ')
    parts = ra_str.split()

    hh = parts[0].zfill(2)  # Ensure 2 digits with leading zero
    mm = parts[1].zfill(2) if len(parts) > 1 else '00'
    ss = parts[2] if len(parts) > 2 else '00.000'

    # Ensure seconds have at least 2 digits before decimal
    if '.' in ss:
        ss_int, ss_frac = ss.split('.')
        ss = ss_int.zfill(2) + '.' + ss_frac
    else:
        ss = ss.zfill(2) + '.000'

    return f"{hh}{mm}{ss}"

def normalize_dec_format(dec_str: str) -> str:
    """
    Convert Dec from various formats to space-separated ±DD MM SS.S format.

    Preserves maximum precision and sign from original MPC data.

    Args:
        dec_str: Declination in format "±DD:MM:SS.S" or "±DD MM SS.S"

    Returns:
        Dec in format "±DD MM SS.SS" (J2000)
    """
    dec_str = dec_str.strip().replace(':', ' ')

    # Handle sign
    sign = '-' if dec_str.startswith('-') else '+'
    dec_str = dec_str.lstrip('+-')

    parts = dec_str.split()
    dd = parts[0].zfill(2)  # Ensure 2 digits with leading zero
    mm = parts[1].zfill(2) if len(parts) > 1 else '00'
    ss = parts[2] if len(parts) > 2 else '00.00'

    # Ensure seconds have at least 2 digits before decimal
    if '.' in ss:
        ss_int, ss_frac = ss.split('.')
        ss = ss_int.zfill(2) + '.' + ss_frac
    else:
        ss = ss.zfill(2) + '.00'

    return f"{sign}{dd} {mm} {ss}"

def parse_observatory_field(obs_field: str) -> tuple:
    """
    Parse MPC observatory field which contains code + name.

    MPC format: "XXX – Observatory Name"
    where XXX is the 3-character MPC observatory code.

    Args:
        obs_field: Full observatory field from MPC

    Returns:
        Tuple of (code, name) where code is 3-char string (preserves leading zeros)
    """
    obs_field = obs_field.strip()

    # Split on the dash separator
    if '–' in obs_field:  # En-dash
        parts = obs_field.split('–', 1)
    elif '-' in obs_field:  # Regular dash
        parts = obs_field.split('-', 1)
    else:
        # No separator, assume it's just the code
        return obs_field[:3].strip(), ""

    code = parts[0].strip()
    name = parts[1].strip() if len(parts) > 1 else ""

    return code, name

def parse_observation_date(date_str: str) -> datetime:
    """
    Parse MPC observation date to datetime object.

    Preserves full subsecond precision from MPC data.

    Args:
        date_str: Date in format "YYYY MM DD.ddddd" (UTC)

    Returns:
        datetime object in UTC with microsecond precision
    """
    parts = date_str.strip().split()
    year = int(parts[0])
    month = int(parts[1])
    day_decimal = float(parts[2])

    day = int(day_decimal)
    fraction = day_decimal - day

    dt = datetime(year, month, day)
    dt += timedelta(days=fraction)

    return dt

print("✅ Helper functions defined (preserving original sexagesimal format)")

✅ Helper functions defined (preserving original sexagesimal format)


## Fetch Object Summary from MPC

In [4]:
def get_object_summary():
    """
    Fetch 3I/ATLAS object summary and total observation count from MPC.
    """
    print(f"🔍 Fetching object summary from MPC...")
    print(f"    URL: {MPC_URL}")

    response = requests.get(MPC_URL, headers=HEADERS, timeout=30)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'lxml')

    # Extract object name from page title or header
    object_name = "3I/ATLAS"  # Default fallback

    # Try multiple methods to find the object name
    # Method 1: Look for h1 or h2 headers
    for header in soup.find_all(['h1', 'h2', 'h3']):
        header_text = header.text.strip()
        if '3I' in header_text or 'ATLAS' in header_text:
            object_name = header_text
            break

    # Method 2: Look in page title
    if object_name == "3I/ATLAS":
        title = soup.find('title')
        if title and ('3I' in title.text or 'ATLAS' in title.text):
            # Extract just the object designation
            title_text = title.text.strip()
            if '3I' in title_text:
                object_name = title_text.split('-')[0].strip() if '-' in title_text else title_text

    # Extract observation count
    obs_count = "Unknown"

    # Method 1: Look for text containing "observation" or "obs"
    for text in soup.stripped_strings:
        # Pattern: "X observations" or "Observations: X" or "Total: X"
        if 'observation' in text.lower():
            match = re.search(r'(\d+)\s*observation', text, re.IGNORECASE)
            if match:
                obs_count = match.group(1)
                break
        # Pattern: "Total: X" or "Count: X"
        if 'total' in text.lower() or 'count' in text.lower():
            match = re.search(r'(\d+)', text)
            if match:
                potential_count = int(match.group(1))
                # Sanity check: observation count should be > 10 and < 100000
                if 10 < potential_count < 100000:
                    obs_count = str(potential_count)
                    break

    # Method 2: Count rows in observation table as fallback
    if obs_count == "Unknown":
        tables = soup.find_all('table')
        for table in tables:
            # Look for table with observation headers
            headers = table.find_all('th')
            if any('Date' in th.text for th in headers):
                # Count data rows (exclude header)
                data_rows = len(table.find_all('tr')) - 1
                if data_rows > 0:
                    obs_count = f"{data_rows} (from table)"
                break

    print(f"\\n{'='*70}")
    print(f"OBJECT CONFIRMATION")
    print(f"{'='*70}")
    print(f"Name:               {object_name}")
    print(f"Total Observations: {obs_count}")
    print(f"Coordinate System:  J2000.0 (ICRF equatorial)")
    print(f"{'='*70}\\n")

    return object_name, obs_count

# Execute
obj_name, total_obs = get_object_summary()

🔍 Fetching object summary from MPC...
    URL: https://minorplanetcenter.net/db_search/show_object?utf8=%E2%9C%93&object_id=3I
\n======================================================================
OBJECT CONFIRMATION
Name:               3I/ATLAS = C/2025 N1 (ATLAS)
Total Observations: 5816
Coordinate System:  J2000.0 (ICRF equatorial)
======================================================================\n


## User Input: Date Range Selection

In [5]:
# Default values
DEFAULT_START = "2025-07-15"  # Approximate discovery date range
DEFAULT_DAYS = 14

print("\n📅 DATE RANGE SELECTION")
print("=" * 70)
print("Enter the date range for observations to scrape.")
print(f"Default start: {DEFAULT_START}")
print(f"Recommended max span: {DEFAULT_DAYS} days per run\n")

# Get user input
start_date_str = input(f"Start date (YYYY-MM-DD) [{DEFAULT_START}]: ").strip()
if not start_date_str:
    start_date_str = DEFAULT_START

days_str = input(f"Number of days to scrape [{DEFAULT_DAYS}]: ").strip()
if not days_str:
    num_days = DEFAULT_DAYS
else:
    num_days = int(days_str)

# Parse dates
start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
end_date = start_date + timedelta(days=num_days)

print(f"\n✅ Selected Range:")
print(f"   Start: {start_date.strftime('%Y-%m-%d')}")
print(f"   End:   {end_date.strftime('%Y-%m-%d')}")
print(f"   Span:  {num_days} days")
print("=" * 70)


📅 DATE RANGE SELECTION
Enter the date range for observations to scrape.
Default start: 2025-07-15
Recommended max span: 14 days per run

Start date (YYYY-MM-DD) [2025-07-15]: 2025-12-17
Number of days to scrape [14]: 16

✅ Selected Range:
   Start: 2025-12-17
   End:   2026-01-02
   Span:  16 days


In [6]:
# Calculate timing estimate
estimate_min = num_days * 0.5
estimate_max = num_days * 1.0

print(f"\n⏱️  TIMING ESTIMATE")
print(f"{'='*70}")
print(f"For {num_days} days of observations:")
print(f"  Expected scraping time: {estimate_min:.1f} - {estimate_max:.1f} minutes")
print(f"  Approximate rate: ~30-60 seconds per day")
print(f"")
print(f"⚠️  The scraping cell may appear frozen for several minutes.")
print(f"    This is NORMAL! Progress messages will appear as parsing completes.")
print(f"    Larger date ranges = longer wait times.")
print(f"{'='*70}\n")


⏱️  TIMING ESTIMATE
For 16 days of observations:
  Expected scraping time: 8.0 - 16.0 minutes
  Approximate rate: ~30-60 seconds per day

⚠️  The scraping cell may appear frozen for several minutes.
    This is NORMAL! Progress messages will appear as parsing completes.
    Larger date ranges = longer wait times.



## Scrape Observations from MPC Table

**Note**: This scrapes the HTML observation table. Each row contains:
- Date (UT) in YYYY MM DD.ddddd format
- J2000 RA in HH:MM:SS.SS format
- J2000 Dec in ±DD:MM:SS.S format
- Magnitude (optional)
- Observatory code (3-character MPC designation)
- Reference (MPEC identifier)

---

### ⏱️ Expected Duration
- **Initial page load**: 2-5 seconds (polite delay to respect MPC servers)
- **Parsing time**: Approximately **20-40 seconds per day** of observations
- **For 14-day range**: Expect **5-10 minutes total**
- **Actual time varies** based on:
  - Network speed
  - MPC server response time
  - Number of observations in date range
  - HTML table complexity

**Don't panic if it seems frozen!** The MPC page can be large (5000+ observations). Progress messages will appear as parsing completes.
---

### 🎯 Precision & Format Guarantees

The scraper ensures:
- **Timestamps**: `YYYY-MM-DD HH:MM:SS.ssssss` format
  - Zero-padded hours: `00-23` (NOT `0-23`)
  - Microsecond precision: 6 decimal places
  - Python's `strftime('%H:%M:%S.%f')` automatically provides this
- **Observatory codes**: 3-character strings (e.g., `"033", "W50"`)
  - Leading zeros explicitly preserved via string dtype
- **RA format**: `HHMMSS.SSS` (packed, zero-padded)
  - Example: `104713.073` = 10h 47m 13.073s (NOT `14713.073`)
- **Dec format**: `±DD MM SS.SS` (space-separated, zero-padded)
  - Example: `+07 00 13.39` (NOT `+7 0 13.39`)


In [7]:
def scrape_observations(start_dt: datetime, end_dt: datetime) -> List[Dict]:
    """
    Scrape MPC observations for 3I/ATLAS within specified date range.

    Preserves original sexagesimal coordinate format and subsecond timestamp precision.

    Returns:
        List of observation dictionaries with J2000 coordinates in original format
    """
    num_days = (end_dt - start_dt).days
    estimated_minutes = num_days * 0.5  # Rough estimate: ~30 sec/day = 0.5 min/day

    print(f"\\n🌐 Scraping observations from MPC...")
    print(f"⏱️  Estimated time: {estimated_minutes:.1f}-{estimated_minutes*2:.1f} minutes for {num_days} days")
    print(f"    (Network speed and server response may vary)")
    print(f"    This is normal - the page is large! Please wait...\\n")

    time.sleep(2)  # Polite delay
    print(f"    ⬇️  Fetching MPC page...")

    response = requests.get(MPC_URL, headers=HEADERS, timeout=30)
    response.raise_for_status()
    print(f"    ✅ Page downloaded ({len(response.content)//1024} KB)")

    print(f"    🔍 Parsing HTML table...")
    soup = BeautifulSoup(response.content, 'lxml')

    # Find observation table
    tables = soup.find_all('table')
    obs_table = None

    for table in tables:
        # Look for table with observation headers
        headers = table.find_all('th')
        if any('Date' in th.text and 'RA' in str(table) for th in headers):
            obs_table = table
            break

    if not obs_table:
        raise ValueError("Could not find observation table on MPC page")

    print(f"    📊 Processing observations...")
    observations = []
    rows = obs_table.find_all('tr')[1:]  # Skip header row
    total_rows = len(rows)
    print(f"    Found {total_rows} total observation rows in table")

    processed = 0
    for row in rows:
        cols = row.find_all('td')
        if len(cols) < 5:
            continue

        try:
            # Parse observation date (preserves microsecond precision)
            date_str = cols[0].text.strip()
            obs_dt = parse_observation_date(date_str)

            # Filter by date range
            if not (start_dt <= obs_dt < end_dt):
                continue

            # Extract fields
            ra_str = cols[1].text.strip()  # Keep in original format
            dec_str = cols[2].text.strip()  # Keep in original format
            mag_str = cols[3].text.strip()
            obs_field = cols[4].text.strip()  # Observatory code + name
            reference = cols[5].text.strip() if len(cols) > 5 else ""

            # Parse observatory field to separate code and name
            obs_code, obs_name = parse_observatory_field(obs_field)

            # Normalize coordinates to standard sexagesimal formats
            ra_formatted = normalize_ra_format(ra_str)  # HHMMSS.SSS
            dec_formatted = normalize_dec_format(dec_str)  # ±DD MM SS.SS

            # Parse magnitude (may be empty)
            magnitude = None
            if mag_str and mag_str not in ['—', '-', '']:
                try:
                    magnitude = float(mag_str)
                except ValueError:
                    pass

            # Format timestamp as YYYY-MM-DD HH:MM:SS.ssssss (preserves microseconds)
            timestamp_str = obs_dt.strftime('%Y-%m-%d %H:%M:%S.%f')

            obs = {
                'timestamp': timestamp_str,
                'observatory': obs_code,  # 3-char code only (preserves leading zeros as string)
                'observatory_name': obs_name,
                'obs_ra': ra_formatted,  # HHMMSS.SSS format (J2000)
                'obs_dec': dec_formatted,  # ±DD MM SS.SS format (J2000)
                'magnitude': magnitude,
                'reference': reference
            }

            observations.append(obs)
            processed += 1

            # Progress indicator every 100 observations
            if processed % 100 == 0:
                print(f"       ... {processed} observations matched so far")

        except Exception as e:
            print(f"⚠️  Warning: Failed to parse row: {e}")
            continue

    print(f"\\n✅ Scraped {len(observations)} observations in date range")
    return observations

# Execute scraping
observations = scrape_observations(start_date, end_date)

\n🌐 Scraping observations from MPC...
⏱️  Estimated time: 8.0-16.0 minutes for 16 days
    (Network speed and server response may vary)
    This is normal - the page is large! Please wait...\n
    ⬇️  Fetching MPC page...
    ✅ Page downloaded (1636 KB)
    🔍 Parsing HTML table...
    📊 Processing observations...
    Found 5816 total observation rows in table
       ... 100 observations matched so far
\n✅ Scraped 193 observations in date range


## Create Output DataFrames

In [8]:
if len(observations) == 0:
    print("\\n⚠️  No observations found in specified date range!")
    print("   Try adjusting the start date or expanding the date range.")
else:
    # Full observation dataset
    df_full = pd.DataFrame(observations)

    # Force observatory code to string type (preserves leading zeros)
    df_full['observatory'] = df_full['observatory'].astype(str)

    # Minimal timestamp/observatory index
    df_index = df_full[['timestamp', 'observatory']].copy()

    # Sort by timestamp
    df_full = df_full.sort_values('timestamp').reset_index(drop=True)
    df_index = df_index.sort_values('timestamp').reset_index(drop=True)

    print(f"\\n📊 DATASET SUMMARY")
    print(f"{'='*70}")
    print(f"Total observations:  {len(df_full)}")
    print(f"Date range:          {df_full['timestamp'].min()} to {df_full['timestamp'].max()}")
    print(f"Observatories:       {df_full['observatory'].nunique()} unique sites")
    print(f"Coordinate system:   J2000.0 equatorial (sexagesimal format)")
    print(f"Coordinate format:   RA=HHMMSS.SSS, Dec=±DD MM SS.SS")
    print(f"{'='*70}")

    # Show sample with format verification
    print("\\n📋 Sample observations (first 5):")
    print("\\nFormat check - verify:")
    print("  - Observatory codes are 3-char strings (with leading zeros if applicable)")
    print("  - RA in packed format HHMMSS.SSS (no spaces)")
    print("  - Dec in space-separated format ±DD MM SS.SS")
    print("  - Timestamps include microsecond precision (.ssssss)\\n")
    print(df_full[['timestamp', 'observatory', 'obs_ra', 'obs_dec']].head())

    print("\\n📋 Observatory distribution (top 10):")
    print(df_full['observatory'].value_counts().head(10))

\n📊 DATASET SUMMARY
Total observations:  193
Date range:          2025-12-17 00:09:07.776000 to 2025-12-23 03:25:15.888000
Observatories:       40 unique sites
Coordinate system:   J2000.0 equatorial (sexagesimal format)
Coordinate format:   RA=HHMMSS.SSS, Dec=±DD MM SS.SS
\n📋 Sample observations (first 5):
\nFormat check - verify:
  - Observatory codes are 3-char strings (with leading zeros if applicable)
  - RA in packed format HHMMSS.SSS (no spaces)
  - Dec in space-separated format ±DD MM SS.SS
  - Timestamps include microsecond precision (.ssssss)\n
                    timestamp observatory     obs_ra      obs_dec
0  2025-12-17 00:09:07.776000         L16  105626.71  +06 13 01.4
1  2025-12-17 00:39:28.224000         L16  105620.91  +06 13 31.3
2  2025-12-17 01:09:51.264000         L16  105615.11  +06 14 00.9
3  2025-12-17 01:45:20.505600         Z92  105608.38  +06 14 35.2
4  2025-12-17 01:56:31.488000         Z92  105606.23  +06 14 45.9
\n📋 Observatory distribution (top 10):
obse

## Save Output Files

In [9]:
if len(observations) > 0:
    # Generate filenames with date range and version tag
    date_suffix = f"{start_date.strftime('%Y%m%d')}_{end_date.strftime('%Y%m%d')}"
    version_tag = "4a7"  # Version identifier for this scraper

    filename_full = f"observations_MPEC_{date_suffix}_v{version_tag}.csv"
    filename_index = f"observations_timestamp_observatory_only_{date_suffix}_v{version_tag}.csv"

    # Create full dataset DataFrame
    # NOTE: Observatory code MUST be dtype=str to preserve leading zeros (e.g., "033")
    df_full = pd.DataFrame(observations)
    df_full['observatory'] = df_full['observatory'].astype(str)  # Force string type

    # Create minimal timestamp/observatory index
    df_index = df_full[['timestamp', 'observatory']].copy()
    df_index['observatory'] = df_index['observatory'].astype(str)  # Force string type

    # Sort by timestamp
    df_full = df_full.sort_values('timestamp').reset_index(drop=True)
    df_index = df_index.sort_values('timestamp').reset_index(drop=True)

    # Save CSVs with string dtype specification for observatory codes
    # This ensures leading zeros are preserved when re-opening
    df_full.to_csv(filename_full, index=False)
    df_index.to_csv(filename_index, index=False)

    print(f"\\n💾 FILES SAVED")
    print(f"{'='*70}")
    print(f"Full dataset:  {filename_full}")
    print(f"               ({len(df_full)} rows, {df_full.shape[1]} columns)")
    print(f"               Columns: {list(df_full.columns)}")
    print(f"")
    print(f"Index file:    {filename_index}")
    print(f"               ({len(df_index)} rows, {df_index.shape[1]} columns)")
    print(f"               (For JPL Horizons ephemeris queries)")
    print(f"{'='*70}")

    print("\\n⚠️  IMPORTANT REMINDERS:")
    print("   - Observatory codes are STRINGS (preserves leading zeros like '033')")
    print("   - Timestamps preserve microsecond precision")
    print("   - RA/Dec in original sexagesimal format (HHMMSS.SSS, ±DD MM SS.SS)")
    print("   - DO NOT open in Excel/Sheets - use Python/pandas or text editors")
    print("   - Version tag: v4a7 for tracking scraper versions")

    print("\\n✅ COMPLETE! Download the files from the Colab file browser (left sidebar).")
    print("   Upload to: 3i-atlas-public-data/observations/")
else:
    print("\\n❌ No files created (no observations in range)")

\n💾 FILES SAVED
Full dataset:  observations_MPEC_20251217_20260102_v4a7.csv
               (193 rows, 7 columns)
               Columns: ['timestamp', 'observatory', 'observatory_name', 'obs_ra', 'obs_dec', 'magnitude', 'reference']

Index file:    observations_timestamp_observatory_only_20251217_20260102_v4a7.csv
               (193 rows, 2 columns)
               (For JPL Horizons ephemeris queries)
\n⚠️  IMPORTANT REMINDERS:
   - Observatory codes are STRINGS (preserves leading zeros like '033')
   - Timestamps preserve microsecond precision
   - RA/Dec in original sexagesimal format (HHMMSS.SSS, ±DD MM SS.SS)
   - DO NOT open in Excel/Sheets - use Python/pandas or text editors
   - Version tag: v4a7 for tracking scraper versions
\n✅ COMPLETE! Download the files from the Colab file browser (left sidebar).
   Upload to: 3i-atlas-public-data/observations/


In [10]:
# ========================================================================
# FORMAT VERIFICATION - Proves the CSV file is correctly formatted
# ========================================================================

if len(observations) > 0:
    print("\n" + "="*70)
    print("🔍 FORMAT VERIFICATION (Character-by-Character Check)")
    print("="*70)

    # Re-read the saved file to verify what was actually written
    df_verify = pd.read_csv(filename_full, dtype={'observatory': str})

    print("\n📅 TIMESTAMP VERIFICATION:")
    for idx, ts in enumerate(df_verify['timestamp'].head(3)):
        hour_part = ts.split(' ')[1].split(':')[0]
        subsec_part = ts.split('.')[1] if '.' in ts else "MISSING"

        print(f"\n  Row {idx+1}: {ts}")
        print(f"    Hour: '{hour_part}' (length={len(hour_part)})  {'✅ Zero-padded' if len(hour_part)==2 else '❌ NOT zero-padded'}")
        print(f"    Subseconds: .{subsec_part}  {'✅ Has precision' if subsec_part != 'MISSING' else '❌ No precision'}")

    print("\n🏛️ OBSERVATORY CODE VERIFICATION:")
    # Check first few observatory codes
    for idx, obs in enumerate(df_verify['observatory'].head(5)):
        has_leading_zero = obs[0] == '0' if len(obs) > 0 else False
        print(f"  Row {idx+1}: '{obs}' (type={type(obs).__name__}, length={len(obs)})  {'✅ Leading zero' if has_leading_zero else '   No leading zero (may be normal)'}")

    print("\n🌟 RA FORMAT VERIFICATION:")
    for idx, ra in enumerate(df_verify['obs_ra'].head(3)):
        ra_str = str(ra)
        print(f"  Row {idx+1}: {ra_str}  (First char: '{ra_str[0]}')  {'✅ Zero-padded hours' if ra_str[0] in '01' else '   (hours >= 10)'}")

    print("\n" + "="*70)
    print("CONCLUSION:")
    print("="*70)

    # Count verifications
    all_hours_padded = all(len(str(ts).split(' ')[1].split(':')[0]) == 2 for ts in df_verify['timestamp'])
    all_have_subsec = all('.' in str(ts) for ts in df_verify['timestamp'])

    if all_hours_padded:
        print("✅ ALL timestamps have zero-padded hours (HH format)")
    else:
        print("❌ Some timestamps missing zero-padded hours")

    if all_have_subsec:
        print("✅ ALL timestamps have subsecond precision (.ssssss)")
    else:
        print("❌ Some timestamps missing subsecond precision")

    print(f"✅ Observatory codes saved as strings (type: {type(df_verify['observatory'].iloc[0]).__name__})")

    print("\n⚠️  IF YOU SEE DIFFERENT RESULTS IN EXCEL/GOOGLE SHEETS:")
    print("   → The CSV file itself IS correct (as verified above)")
    print("   → The SPREADSHEET is destroying the visual display")
    print("   → Use Python/pandas, text editors, or import as TEXT columns")
    print("="*70)


🔍 FORMAT VERIFICATION (Character-by-Character Check)

📅 TIMESTAMP VERIFICATION:

  Row 1: 2025-12-17 00:09:07.776000
    Hour: '00' (length=2)  ✅ Zero-padded
    Subseconds: .776000  ✅ Has precision

  Row 2: 2025-12-17 00:39:28.224000
    Hour: '00' (length=2)  ✅ Zero-padded
    Subseconds: .224000  ✅ Has precision

  Row 3: 2025-12-17 01:09:51.264000
    Hour: '01' (length=2)  ✅ Zero-padded
    Subseconds: .264000  ✅ Has precision

🏛️ OBSERVATORY CODE VERIFICATION:
  Row 1: 'L16' (type=str, length=3)     No leading zero (may be normal)
  Row 2: 'L16' (type=str, length=3)     No leading zero (may be normal)
  Row 3: 'L16' (type=str, length=3)     No leading zero (may be normal)
  Row 4: 'Z92' (type=str, length=3)     No leading zero (may be normal)
  Row 5: 'Z92' (type=str, length=3)     No leading zero (may be normal)

🌟 RA FORMAT VERIFICATION:
  Row 1: 105626.71  (First char: '1')  ✅ Zero-padded hours
  Row 2: 105620.91  (First char: '1')  ✅ Zero-padded hours
  Row 3: 105615.11  (F

---

## 📚 Reference: Column Definitions

### Full Dataset (`observations_MPEC_{dates}.csv`)

| Column | Type | Format | Description |
|--------|------|--------|-------------|
| `timestamp` | **string** | `YYYY-MM-DD HH:MM:SS.ssssss` | UTC observation time with microsecond precision |
| `observatory` | **string** | `XXX` (3 chars) | MPC observatory code (e.g., "W50", "033", "G96") - **MUST be string to preserve leading zeros** |
| `observatory_name` | string | Free text | Full observatory name (e.g., "Whipple Observatory") |
| `obs_ra` | string | `HHMMSS.SSS` | Right Ascension in packed sexagesimal format (J2000.0) - **NO spaces, NO colons** |
| `obs_dec` | string | `±DD MM SS.SS` | Declination in space-separated sexagesimal format (J2000.0) |
| `magnitude` | float | Decimal | Apparent visual magnitude (nullable) |
| `reference` | string | MPEC code | MPEC reference identifier (e.g., "MPEC Y51") |

### Index File (`observations_timestamp_observatory_only_{dates}.csv`)

| Column | Type | Format | Description |
|--------|------|--------|-------------|
| `timestamp` | **string** | `YYYY-MM-DD HH:MM:SS.ssssss` | UTC observation time |
| `observatory` | **string** | `XXX` (3 chars) | MPC observatory code - **MUST be string** |

**Purpose**: This minimal file contains only the two fields needed for JPL Horizons observer-specific ephemeris queries.

---

## 🔍 Format Examples

### ✅ CORRECT Examples:
```csv
timestamp,observatory,obs_ra,obs_dec
2025-12-19 00:10:29.123456,B67,104713.073,+07 00 13.39
2025-12-19 00:37:56.789012,D69,104707.680,+07 00 40.32
2025-12-19 01:43:00.000000,033,104655.022,+07 01 44.33
```

**Note**:
- Timestamp: Zero-padded hours (`00:10:29`, `01:43:00`) + microsecond precision (`.123456`)
- Observatory: `"033"` preserves leading zero (string type)
- RA: Zero-padded hours in packed format (`104713.073` = 10h 47m 13.073s)
- Dec: Zero-padded degrees (`+07` not `+7`), space-separated
timestamp,observatory,obs_ra,obs_dec
2025-12-19 00:10:29.000000,B67,104713.073,+07 00 13.39
2025-12-19 00:37:56.000000,D69,104707.680,+07 00 40.32
2025-12-19 01:43:00.000000,033,104655.022,+07 01 44.33

**Note**: Observatory "033" preserves leading zero (string type), RA has NO spaces (packed format).


### ❌ INCORRECT Examples (what NOT to do):
```csv
# WRONG: Missing leading zero on hours
2025-12-19 0:10:29.123456,B67,104713.073,+07 00 13.39  # Should be 00:10:29

# WRONG: Missing subsecond precision
2025-12-19 00:10:29,B67,104713.073,+07 00 13.39  # Should have .ssssss

# WRONG: Observatory code as number (loses leading zeros)
2025-12-19 01:43:00.000000,33,104655.022,+07 01 44.33  # Should be "033"

# WRONG: RA in decimal degrees (not sexagesimal)
2025-12-19 00:10:29.123456,B67,161.804471,+7.003719

# WRONG: RA with spaces or colons
2025-12-19 00:10:29.123456,B67,10 47 13.073,+07 00 13.39
2025-12-19 00:10:29.123456,B67,10:47:13.073,+07 00 13.39

# WRONG: RA missing leading zero on hours
2025-12-19 00:10:29.123456,B67,14713.073,+07 00 13.39  # Should be 104713.073
```
```csv
# WRONG: Observatory code as number (loses leading zeros)
2025-12-19 01:43:00,33,104655.022,+07 01 44.33  # Should be "033"

# WRONG: RA in decimal degrees (not sexagesimal)
2025-12-19 00:10:29,B67,161.804471,+7.003719

# WRONG: RA with spaces or colons
2025-12-19 00:10:29,B67,10 47 13.073,+07 00 13.39
2025-12-19 00:10:29,B67,10:47:13.073,+07 00 13.39

# WRONG: Timestamp truncated (lost subsecond precision)
2025-12-19 00:10:29,B67,104713.073,+07 00 13.39  # Missing .000000
```

---

## ⚠️ Critical Data Handling Rules

### When Reading These Files:
```python
# ✅ CORRECT: Force string dtypes for observatory codes
import pandas as pd
df = pd.read_csv('observations_MPEC.csv', dtype={'observatory': str})

# ❌ WRONG: Auto-detect types (will lose leading zeros)
df = pd.read_csv('observations_MPEC.csv')  # Observatory "033" becomes 33
```

### When Writing These Files:
```python
# ✅ CORRECT: Explicitly set string dtype before saving
df['observatory'] = df['observatory'].astype(str)
df.to_csv('output.csv', index=False)
```

### In Spreadsheet Software:
**⚠️ AVOID if possible! If you MUST use Excel/Google Sheets:**
1. Import as CSV (not open directly)
2. Specify ALL columns as "Text" format during import
3. Never use auto-detect
4. Verify leading zeros are preserved

---

## 🔗 Related Documentation

- **MPC Observatory Codes**: https://minorplanetcenter.net/iau/lists/ObsCodesF.html
- **MPEC Format Guide**: https://minorplanetcenter.net/iau/info/MPCFormat.html
- **J2000 Reference Frame**: https://aa.usno.navy.mil/faq/ICRS_doc
- **JPL Horizons System**: https://ssd.jpl.nasa.gov/horizons/
- **3I/ATLAS Discovery**: MPEC 2025-N12

---

**Repository**: https://github.com/l-87hjl/3i-atlas-public-data  
**License**: CC0 1.0 (Public Domain)
